In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath("../src"))

In [2]:
import numpy             as np
import matplotlib.pyplot as pl
import pandas            as pd
import seaborn           as sb

# The `tyche` package is located at <https://github.com/NREL/portfolio/tree/master/production-function/src/tyche/>.
import tyche             as ty

from copy            import deepcopy

In [3]:
designs = ty.Designs("data/pv_residential_simple")

In [4]:
investments = ty.Investments("data/pv_residential_simple")

In [5]:
designs.compile()

In [6]:
tranche_results = investments.evaluate_tranches(designs, sample_count=50)

In [7]:
tranche_results.amounts

Amount
Category     Tranche                       
BoS R&D      BoS High R&D          900000.0
             BoS Low R&D           300000.0
             BoS Medium R&D        600000.0
Inverter R&D Inverter High R&D    3000000.0
             Inverter Low R&D     1000000.0
             Inverter Medium R&D  2000000.0
Module R&D   Module High R&D      4500000.0
             Module Low R&D       1500000.0
             Module Medium R&D    3000000.0

In [8]:
evaluator = ty.Evaluator(investments.tranches, tranche_results.summary)

In [9]:
optimizer = ty.EpsilonConstraintOptimizer(evaluator)

In [10]:
metric_max = optimizer.max_metrics()
metric_max

GHG      50.040528
LCOE      0.062988
Labor    49.072317
Name: Value, dtype: float64

In [11]:
investment_max = 3e6

In [12]:
metric_min = pd.Series([40, 0], name = "Value", index = ["GHG", "Labor"])
metric_min

GHG      40
Labor     0
Name: Value, dtype: int64

In [13]:
# Use SLSQP.

In [14]:
optimum = optimizer.maximize_slsqp(
    "LCOE"                       ,
    total_amount = investment_max,
    min_metric   = metric_min    ,
    statistic    = np.mean       ,
)
optimum.exit_message

'Optimization terminated successfully.'

In [15]:
np.round(optimum.amounts)

Category
BoS R&D           75521.0
Inverter R&D          0.0
Module R&D      2924479.0
Name: Amount, dtype: float64

In [16]:
optimum.metrics

Index
GHG      4.228141e+01
LCOE     3.571137e-02
Labor   -2.480821e-12
Name: Value, dtype: float64

In [17]:
# Use Piecewise-Linear MILP.

In [18]:
optimum = optimizer.pwlinear_milp(
    "LCOE"                       ,
    total_amount = investment_max,
    min_metric   = metric_min    ,
    statistic    = np.mean       ,
)
optimum.exit_message

<OptimizationStatus.INFEASIBLE: 1>

In [ ]:
np.round(optimum.amounts)

In [ ]:
optimum.metrics